In [2]:
import openpyxl
from openpyxl.styles import Alignment
import requests
from bs4 import BeautifulSoup

In [3]:

def detailinforequest(rrdcNo):
# URL của API
    url = 'https://unipass.customs.go.kr/clip/prlstclsfsrch/retrieveDmstPrlstClsfCaseDtl.do'

    # Headers
    headers = {
        'Accept': 'text/html, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 'WMONID=N_4A3dINH0I; JSESSIONID=0001O2y8S-yyQQ_lga-knblPV6Lb2QN6662ivRFJ_jK2FPRlLOTKNEsnuhZe_QxBEKq-8U03_kah8EgUgb-_Exa4dmevvhJHEKeNuIq9prRxpJht6ugFASENgz3p_AdMPw4_:eul21',
        'DNT': '1',
        'Origin': 'https://unipass.customs.go.kr',
        'Referer': 'https://unipass.customs.go.kr/clip/index.do',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'isAjax': 'true',
        'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"'
    }

    # Dữ liệu trong body của yêu cầu
    payload = {
        'rrdcNo': rrdcNo
    }

    # Gửi yêu cầu POST với headers và dữ liệu body
    response = requests.post(url, headers=headers, data=payload)
    
    # Kiểm tra phản hồi từ API
    if response.status_code == 200:
        return response.text  
    else:
        print(rrdcNo + "Lỗi")



In [4]:


# Hàm trích xuất các thành phần từ HTML
def extract_data_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Tìm các thành phần theo nhãn <tr> và trích xuất giá trị
    ref_no = soup.find('th', text='참조번호').find_next_sibling('td').text.strip()
    implementation_date = soup.find('th', text='시행일자').find_next_sibling('td').text.strip()
    implementing_agency = soup.find('th', text='시행기관').find_next_sibling('td').text.strip()
    
    # Chỉ lấy phần đầu tiên của quyết định (trước dấu xuống dòng \n)
    decision_code = soup.find('th', text='결정세번').find_next_sibling('td').find('input')['value'].strip()

    product_name = soup.find('th', text='품명').find_next_sibling('td').text.strip()
    product_description = soup.find('th', text='물품설명').find_next_sibling('td').text.strip()
    decision_reason = soup.find('th', text='결정사유').find_next_sibling('td').text.strip()
    image_count = soup.find('th', text='이미지건수').find_next_sibling('td').text.strip()
    
    # Liệt kê các hình ảnh trong mục "관련 이미지"
    image_elements = soup.find('th', text='관련 이미지').find_next_sibling('td').find_all('img')
    image_links = [img['src'] for img in image_elements]
    
    # Tạo kết quả dưới dạng dictionary
    result = {
        "참조번호": ref_no,
        "시행일자": implementation_date,
        "시행기관": implementing_agency,
        "결정세번": decision_code,
        "품명": product_name,
        "물품설명": product_description,
        "결정사유": decision_reason,
        "이미지건수": image_count,
        "관련 이미지": image_links
    }
    
    return result


In [4]:

# Ghi tiêu đề
# headers = list(data.keys())
# ws.append(headers)

In [27]:
def writedatatoexcel(data):
    # Tìm dòng tiếp theo trống
    next_row = ws.max_row + 1  # Dòng tiếp theo trống

    # Ghi dữ liệu
    row_data = []
    for key, value in data.items():
        if isinstance(value, list):  # Nếu là list, gộp thành chuỗi
            value = ', '.join(value)
        row_data.append(value.replace('\r', '\n'))  # Thay \r bằng \n để xuống dòng

    # Ghi dữ liệu vào dòng tiếp theo
    for col, val in enumerate(row_data, start=1):
        cell = ws.cell(row=next_row, column=col, value=val)
        cell.alignment = Alignment(wrap_text=True)  # Bật tính năng Wrap Text cho từng ô

    # Điều chỉnh độ rộng cột
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter  # Lấy chữ cái cột
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column_letter].width = adjusted_width

    # Lưu file Excel
    wb.save(file_name)
    print(f"Data saved to {file_name}.")


In [5]:
# Hàm thực hiện gọi API và xử lý phản hồi
def call_api(prlstClsfCaseTpcd_list, dtrmHsSgn_range):
    # URL của API
    url = "https://unipass.customs.go.kr/clip/prlstclsfsrch/retrieveDmstPrlstClsfCaseLst2.do"
    
    # Headers của yêu cầu
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7,fr-FR;q=0.6,fr;q=0.5',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 'WMONID=PrQbL04HN94; JSESSIONID=0001QEkyah0hPybS5B8l72lu5sN66qPDua8EuJLATsU9U2O4a48A8opBefekkxF-oK6X35ZbCYaclUatspLmr_JtTHoTUlk6y_EdohZShZVj65_39JPhQ4_pBRGAXPEo0vry:eul11',
        'Origin': 'https://unipass.customs.go.kr',
        'Referer': 'https://unipass.customs.go.kr/clip/index.do',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'isAjax': 'true',
        'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"'
    }

    # Tạo các danh sách để chứa kết quả cho từng giá trị prlstClsfCaseTpcd
    prlstClsfCaseTpcd_results = {value: [] for value in prlstClsfCaseTpcd_list}
    
    # Lặp qua các giá trị dtrmHsSgn và prlstClsfCaseTpcd
    for dtrmHsSgn in dtrmHsSgn_range:
        for prlstClsfCaseTpcd in prlstClsfCaseTpcd_list:
            # Dữ liệu body của yêu cầu
            data = {
                'dtrmHsSgn': str(dtrmHsSgn),
                'prlstClsfCaseTpcd': prlstClsfCaseTpcd,
                'pageUnit': '5000'
            }
            
            # Thực hiện yêu cầu POST
            response = requests.post(url, headers=headers, data=data)
            
            # Kiểm tra và in kết quả phản hồi theo định dạng JSON
            try:
                json_response = response.json()
                total_records = json_response["paginationInfo"]["totalRecordCount"]
                print(f"Total records for dtrmHsSgn={dtrmHsSgn}, prlstClsfCaseTpcd={prlstClsfCaseTpcd}: {total_records}")
                
                # Duyệt qua từng mục trong "itemList" và lưu "RRDC_NO" vào danh sách tương ứng
                for i in range(total_records):
                    rrdc_no = json_response["uls_dmst"]["itemList"][i]["RRDC_NO"]
                    prlstClsfCaseTpcd_results[prlstClsfCaseTpcd].append(rrdc_no)
                    
            except (ValueError, KeyError) as e:
                print(f"Phản hồi lỗi cho dtrmHsSgn={dtrmHsSgn}, prlstClsfCaseTpcd={prlstClsfCaseTpcd}: {e}")
                print(response.text)
    
    # Trả về kết quả cho từng danh sách
    return prlstClsfCaseTpcd_results




In [7]:
prlstClsfCaseTpcd_values = ['04', '03', '01']
dtrmHsSgn_values = range(100, 1000, 10)

In [8]:
rrdcNo_lists = call_api(prlstClsfCaseTpcd_values, dtrmHsSgn_values)


Total records for dtrmHsSgn=100, prlstClsfCaseTpcd=04: 6
Total records for dtrmHsSgn=100, prlstClsfCaseTpcd=03: 0
Total records for dtrmHsSgn=100, prlstClsfCaseTpcd=01: 188
Total records for dtrmHsSgn=110, prlstClsfCaseTpcd=04: 3
Total records for dtrmHsSgn=110, prlstClsfCaseTpcd=03: 4
Total records for dtrmHsSgn=110, prlstClsfCaseTpcd=01: 476
Total records for dtrmHsSgn=120, prlstClsfCaseTpcd=04: 1
Total records for dtrmHsSgn=120, prlstClsfCaseTpcd=03: 2
Total records for dtrmHsSgn=120, prlstClsfCaseTpcd=01: 136
Total records for dtrmHsSgn=130, prlstClsfCaseTpcd=04: 2
Total records for dtrmHsSgn=130, prlstClsfCaseTpcd=03: 3
Total records for dtrmHsSgn=130, prlstClsfCaseTpcd=01: 214
Total records for dtrmHsSgn=140, prlstClsfCaseTpcd=04: 0
Total records for dtrmHsSgn=140, prlstClsfCaseTpcd=03: 4
Total records for dtrmHsSgn=140, prlstClsfCaseTpcd=01: 53
Total records for dtrmHsSgn=150, prlstClsfCaseTpcd=04: 0
Total records for dtrmHsSgn=150, prlstClsfCaseTpcd=03: 0
Total records for dtrm

In [10]:
# txt_file = '01_list.txt'

# # Ghi danh sách vào file
# with open(txt_file, 'w', encoding='utf-8') as file:
#     for item in rrdcNo_lists["01"]:
#         file.write(f"{item}\n")  # Ghi từng phần tử vào file và xuống dòng

# print(f"Danh sách đã được ghi ra file {txt_file}.")

In [7]:
list_file = "01_list.txt"

with open(list_file, 'r', encoding='utf-8') as file:
    content = file.read()  # Đọc toàn bộ nội dung file

# Chia nhỏ theo dấu cách hoặc dấu phân cách khác
data_list = content.split()  # Chia nhỏ theo dấu cách (space)

print(len(data_list))

26586
